## Get data from API

In [1]:
import pydeck as pdk
import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 99999 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
# Get Bedbug Reporting dataset
results = client.get("wz6d-d3jb", limit=99999)

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)
# The api doesn't give us the column data types, so we have to manually assign them to use them with pydeck
df["infested_dwelling_unit_count"]=pd.to_numeric(df["infested_dwelling_unit_count"])
df["latitude"]=pd.to_numeric(df["latitude"])
df["longitude"]=pd.to_numeric(df["longitude"])

In [2]:
df

,building_id,registration_id,borough,house_number,street_name,postcode,of_dwelling_units,infested_dwelling_unit_count,eradicated_unit_count,re_infested_dwelling_unit,...,filing_period_start_date,filling_period_end_date,latitude,longitude,community_board,city_council_district,census_tract_2010,bin,bbl,nta
0,3,138953,MANHATTAN,1006,1 AVENUE,10022,626,0.0,0,0,...,2018-11-01T00:00:00.000,2019-10-31T00:00:00.000,40.757148,-73.963757,106,5,10601,1040460,1013670001,Turtle Bay-East Midtown
1,3,138953,MANHATTAN,1006,1 AVENUE,10022,626,0.0,0,0,...,2017-11-01T00:00:00.000,2018-11-30T00:00:00.000,40.757148,-73.963757,106,5,10601,1040460,1013670001,Turtle Bay-East Midtown
2,7,100463,MANHATTAN,1026,1 AVENUE,10022,253,1.0,1,0,...,2018-11-01T00:00:00.000,2019-10-31T00:00:00.000,40.757782,-73.963294,106,5,10601,1040466,1013680001,Turtle Bay-East Midtown
3,7,100463,MANHATTAN,1026,1 AVENUE,10022,253,3.0,3,0,...,2017-11-01T00:00:00.000,2018-11-30T00:00:00.000,40.757782,-73.963294,106,5,10601,1040466,1013680001,Turtle Bay-East Midtown
4,8,135763,MANHATTAN,103,1 AVENUE,10003,4,4.0,4,4,...,2018-11-01T00:00:00.000,2019-10-31T00:00:00.000,40.726604,-73.986023,103,2,38,1006288,1004480032,East Village
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66125,992422,144683,MANHATTAN,100,VARICK STREET,10013,112,0.0,0,0,...,2017-11-01T00:00:00.000,2018-11-30T00:00:00.000,40.723997,-74.006075,102,3,37,1089738,1004777502,SoHo-TriBeCa-Civic Center-Little Italy
66126,992474,144685,MANHATTAN,269,WEST 87 STREET,10024,38,0.0,0,0,...,2018-11-01T00:00:00.000,2019-10-31T00:00:00.000,40.789248,-73.976602,107,6,175,1090817,1012350007,Upper West Side
66127,992475,144686,MANHATTAN,1861,BROADWAY,10023,171,0.0,0,0,...,2018-11-01T00:00:00.000,2019-10-31T00:00:00.000,40.769866,-73.982133,107,3,145,1090158,1011140009,Lincoln Square
66128,992862,433343,QUEENS,41-53,77 STREET,NaN,8,0.0,0,0,...,2018-11-01T00:00:00.000,2019-10-31T00:00:00.000,40.744418,-73.888140,404,25,267,4617786,4014970051,Elmhurst


## Build and display bedbug map

In [3]:
layer = pdk.Layer(
    "HeatmapLayer",
    data=df,
    get_position=['longitude', 'latitude'],
    get_weight="infested_dwelling_unit_count",
    threshold=0.3)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=-74.0060,
    latitude=40.7128,
    zoom=11,
    min_zoom=5,
    max_zoom=16,
    pitch=45,
    bearing=-27.36)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
#r.show()
r.to_html('demo.html')

# Please see the note about using a Mapbox API token here:
# https://github.com/uber/deck.gl/tree/master/bindings/python/pydeck#mapbox-api-token

'/home/axm/Documents/notebooks/demo.html'

### Now lets see if theres any correlation between population density and bedbug infestations

To make our population density, we will need to combine the map of NYC split into neighborhood tabulation areas and the dataset of the population of those NTAs

In [4]:
# Get New York City Population By Neighborhood Tabulation Areas dataset
results = client.get("swpk-hqdp", limit=2000)
results_df = pd.DataFrame.from_records(results)

In [5]:
# Get Neighborhood Tabulation Areas (NTA) Map
nycmap = client.get("q2z5-ai38", limit=2000, content_type="geojson")

In [6]:
results_df.describe()

,borough,year,fips_county_code,nta_code,nta_name,population
count,390,390,390,390,390,390
unique,5,2,5,195,195,387
top,Queens,2000,081,BK75,Battery Park City-Lower Manhattan,0
freq,116,195,116,2,2,3


The population dataset contains years 2000 and 2010, we're only interested in 2010, so lets drop the rest

In [7]:
results_df_2010 = results_df[results_df['year']== '2010']

In [8]:
results_df_2010.describe()

,borough,year,fips_county_code,nta_code,nta_name,population
count,195,195,195,195,195,195
unique,5,1,5,195,195,193
top,Queens,2010,081,BK75,Battery Park City-Lower Manhattan,0
freq,58,195,58,1,1,2


In [9]:
len(nycmap)

3

Now that our datasets match up, lets combine them

In [10]:
from area import area
import json
# create dictionary of nta codes mapping to area (square miles)
d = {}
neighborhood = nycmap["features"]
for n in neighborhood:
    code = n["properties"]["ntacode"]
    a = area(n["geometry"])/(1609*1609) # converts from m^2 to mi^2
    #n["properties"]["density"] = results_df_2010["population"][results_df_2010["nta_code"] == code]
    y = {"density" : int(results_df_2010["population"][results_df_2010["nta_code"] == code])//a}
    n["properties"].update(y)

In [11]:
nycmap["features"][0]["properties"]["density"]

54823.0

We need to normalize the data to map it to the 255 intensity values for our final map

In [12]:
maxim = 0
minim = float('inf')
for i in nycmap["features"]:
    if i["properties"]["density"] > maxim:
        maxim = i["properties"]["density"]
    if i["properties"]["density"] < minim:
        minim = i["properties"]["density"]
print(maxim)
print(minim)

159518.0
0.0


In [13]:
layer2 = pdk.Layer(
    "GeoJsonLayer",
    data=nycmap,
    pickable=False,
    stroked=False,
    filled=True,
    get_elevation=30,
    wireframe=True,
    opacity=1,
    get_fill_color='[(properties.density / 623), (properties.density / 623), (properties.density / 623)]',
    get_line_color=[255, 255, 255])

# Render
r = pdk.Deck(layers=[layer,layer2], initial_view_state=view_state)
r.to_html('demo.html')
#r.show()

'/home/axm/Documents/notebooks/demo.html'

By overlaying the two layers, we can see that there is a correlation between the population density and the amount of infestations

Finally, lets add a dropdown menu so that we can flip between the layers.

In [14]:
import ipywidgets as widgets
from IPython.display import display

dd = widgets.Dropdown(
    options=['both', 'bedbugs', 'population density'],
    value='both',
    description='Visible Layers:',
    disabled=False,
)

def on_change(v):
    if dd.value == 'both':
        r = pdk.Deck(layers=[layer,layer2], initial_view_state=view_state,mapbox_key='pk.eyJ1IjoiYXhtIiwiYSI6ImNrYmxiZjh3ODE3ZDcycm1sajN0NDk5cDcifQ.3oO6y6On9Ir1G2tQziqXyw')
        r.update()
    if dd.value == 'bedbugs':
        r = pdk.Deck(layers=[layer], initial_view_state=view_state,mapbox_key='pk.eyJ1IjoiYXhtIiwiYSI6ImNrYmxiZjh3ODE3ZDcycm1sajN0NDk5cDcifQ.3oO6y6On9Ir1G2tQziqXyw')
        r.update()
    if dd.value == 'population density':
        r = pdk.Deck(layers=[layer2], initial_view_state=view_state,mapbox_key='pk.eyJ1IjoiYXhtIiwiYSI6ImNrYmxiZjh3ODE3ZDcycm1sajN0NDk5cDcifQ.3oO6y6On9Ir1G2tQziqXyw')
        r.update()
dd.observe(on_change,names = 'value')
display(dd)

Dropdown(description='Visible Layers:', options=('both', 'bedbugs', 'population density'), value='both')

Further exploration may include comparing the wealth distribution of these areas, or how often infestation rates appear in rentals vs owned properties, etc.